In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [32]:
X = pd.read_csv('X_resample_SMOTE.csv')
y = pd.read_csv('y_resample_SMOTE.csv')
test = pd.read_csv('test.csv')

In [33]:
def calculate_woe_iv(data, feature, target, bins=10):
    """
    Calculate Weight of Evidence (WoE) and Information Value (IV) for a given feature.

    Args:
    - data (pd.DataFrame): DataFrame containing the feature and target.
    - feature (str or pd.Series): The feature column name or Series object.
    - target (str): The target column name (binary: 0 or 1).
    - bins (int or list): Number of bins or specific bin edges.

    Returns:
    - woe_df (pd.DataFrame): DataFrame with WoE and IV for each bin.
    - iv (float): Information Value of the feature.
    """
    # Check if `feature` is a column name or Series
    if isinstance(feature, str):
        feature_series = data[feature]
    elif isinstance(feature, pd.Series):
        feature_series = feature
    else:
        raise ValueError("Feature must be either a column name (str) or a pandas Series.")
    
    # Bin the feature
    if isinstance(bins, int):
        data['bin'] = pd.qcut(feature_series, q=bins, duplicates='drop')
    else:
        data['bin'] = pd.cut(feature_series, bins=bins)
    
    # Group by bins and calculate statistics
    grouped = data.groupby('bin').agg(
        total_count=(target, 'count'),
        bad_count=(target, 'sum'),
        good_count=(target, lambda x: x.count() - x.sum())
    ).reset_index()
    
    # Add good and bad percentages
    grouped['bad_pct'] = grouped['bad_count'] / grouped['bad_count'].sum()
    grouped['good_pct'] = grouped['good_count'] / grouped['good_count'].sum()
    
    # Calculate WoE and handle division by zero
    grouped['woe'] = np.log((grouped['good_pct'] + 1e-10) / (grouped['bad_pct'] + 1e-10))
    
    # Calculate IV
    grouped['iv'] = (grouped['good_pct'] - grouped['bad_pct']) * grouped['woe']
    iv = grouped['iv'].sum()
    
    return grouped[['bin', 'woe', 'iv']], iv

In [34]:
features = X.columns

In [35]:
X['TARGET'] = y['TARGET'].values

In [36]:
t = 'TARGET'

In [37]:
feature_iv = {}
for feature in features:
    woe_df, iv = calculate_woe_iv(data=X, feature=feature, target=t, bins=5)
    feature_iv[feature] = iv
    print(f"Feature: {feature}")
    print(woe_df)
    print(f"IV: {iv}\n")

C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(
C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(


Feature: NAME_CONTRACT_TYPE
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0

Feature: CODE_GENDER
               bin       woe        iv
0  (-0.001, 0.425]  0.219703  0.028846
1     (0.425, 1.0] -0.331232  0.043489
IV: 0.07233475857210649



C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(


Feature: FLAG_OWN_CAR
               bin       woe        iv
0  (-0.001, 0.106]  0.187219  0.020969
1     (0.106, 1.0] -0.281864  0.031570
IV: 0.05253966513080642

Feature: FLAG_OWN_REALTY
               bin       woe        iv
0  (-0.001, 0.959] -0.486421  0.092819
1     (0.959, 1.0]  0.320758  0.061208
IV: 0.15402703220336475



C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(
C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(


...

C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(
C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(


Feature: WEEKDAY_APPR_PROCESS_START_MONDAY
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0

Feature: WEEKDAY_APPR_PROCESS_START_SATURDAY
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0



C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(
C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(


Feature: WEEKDAY_APPR_PROCESS_START_SUNDAY
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0

Feature: WEEKDAY_APPR_PROCESS_START_THURSDAY
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0

Feature: WEEKDAY_APPR_PROCESS_START_TUESDAY
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0

Feature: WEEKDAY_APPR_PROCESS_START_WEDNESDAY
             bin  woe   iv
0  (-0.001, 1.0]  0.0  0.0
IV: 0.0



C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(
C:\Users\nn200\AppData\Local\Temp\ipykernel_33640\1823657224.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bin').agg(


In [38]:
sorted_iv = sorted(feature_iv.items(), key=lambda x: x[1], reverse=True)
print("Features sorted by IV:")
for feature, iv in sorted_iv:
    print(f"{feature}: IV = {iv}")

Features sorted by IV:
REGION_RATING_CLIENT: IV = 2.3636952346867446
EMERGENCYSTATE_MODE: IV = 1.731131894077212
REGION_RATING_CLIENT_W_CITY: IV = 1.6117033060280073
EXT_SOURCE_MEAN: IV = 0.762636343120367
EXT_SOURCE_2: IV = 0.4612540833605343
EXT_SOURCE_3: IV = 0.4087721569823295
YEARS_BEGINEXPLUATATION_AVG: IV = 0.35740740657287257
NUM_EXT_SOURCES: IV = 0.32133501341695697
MONTH_DAYS_EMPLOYED: IV = 0.22160602153188547
LIVINGAREA_AVG: IV = 0.1984587258123924
AMT_REQ_CREDIT_BUREAU_QRT: IV = 0.19632127008277717
FLAG_PHONE: IV = 0.18493347109178243
AMT_CREDIT: IV = 0.17973662242094576
MONTH_DAYS_ID_PUBLISH: IV = 0.17178290959707893
ELEVATORS_AVG: IV = 0.1710996902455106
MONTH_DAYS_BIRTH: IV = 0.16625720445466322
FLAG_OWN_REALTY: IV = 0.15402703220336475
HOUR_APPR_PROCESS_START: IV = 0.145323168659649
ANNUITY_LENGTH: IV = 0.1448076437165166
AMT_GOODS_PRICE: IV = 0.142262729978393
OBS_30_CNT_SOCIAL_CIRCLE: IV = 0.13427542202953305
MONTH_DAYS_LAST_PHONE_CHANGE: IV = 0.12839149319649779
OBS_

In [39]:
filtered_features = [feature for feature, iv in feature_iv.items() if 0.1 <= iv <= 0.5]

# Create a new DataFrame with only the filtered features
filtered_train = X[filtered_features]

In [40]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=42)
# filtered_train, y_res = smote.fit_resample(filtered_train, y_res)

In [41]:
filtered_test = test[filtered_features]

In [42]:
filtered_test.columns

Index(['FLAG_OWN_REALTY', 'AMT_CREDIT', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'FLAG_PHONE', 'HOUR_APPR_PROCESS_START',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'YEARS_BEGINEXPLUATATION_AVG',
       'ELEVATORS_AVG', 'LIVINGAREA_AVG', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'MONTH_DAYS_BIRTH', 'MONTH_DAYS_EMPLOYED', 'MONTH_DAYS_REGISTRATION',
       'MONTH_DAYS_ID_PUBLISH', 'MONTH_DAYS_LAST_PHONE_CHANGE',
       'ANNUITY_LENGTH', 'NUM_EXT_SOURCES'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.impute import SimpleImputer  # Thay thế Imputer bằng SimpleImputer

# Khởi tạo các preprocessing. Trong đó SimpleImputer theo mean, MinMaxScaler theo khoảng 0, 1 và PolynomialFeatures bậc 3.
imputer = SimpleImputer(strategy='mean')  # Sửa đổi từ Imputer thành SimpleImputer
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
poly_engineer = PolynomialFeatures(degree=3)

# Lấy các feature có tương quan lớn nhất đến biến mục tiêu từ app_train và app_test
TARGET = y['TARGET']
train_poly_fea = filtered_train
test_poly_fea = filtered_test

In [45]:
# MinMaxScaler dữ liệu
train_poly_fea = minmax_scaler.fit_transform(train_poly_fea)
test_poly_fea = minmax_scaler.transform(test_poly_fea)

In [46]:
# Tạo ra các Polynomial Features
train_poly_fea = poly_engineer.fit_transform(train_poly_fea)
test_poly_fea = poly_engineer.transform(test_poly_fea)

MemoryError: Unable to allocate 8.76 GiB for an array with shape (452266, 2600) and data type float64

In [ ]:
# Tạo ra các Polynomial Features
train_poly_fea = poly_engineer.fit_transform(train_poly_fea)
test_poly_fea = poly_engineer.transform(test_poly_fea)

print('train_poly_fea shape: ', train_poly_fea.shape)
print('test_poly_fea shape: ', test_poly_fea.shape)

MemoryError: Unable to allocate 8.76 GiB for an array with shape (452266, 2600) and data type float64

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'classifier__penalty': ['l2'],
    'classifier__dual': [False],
    'classifier__tol': [1e-3, 1e-4, 1e-5],
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__fit_intercept': [True],
    'classifier__intercept_scaling': [1],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__solver': ['lbfgs', 'newton-cg', 'sag'],
    'classifier__max_iter': [100, 200, 300],
}

model_pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression(random_state=42))
])

random_search = RandomizedSearchCV(
    estimator = model_pipeline,
    param_distributions = param_distributions,
    n_iter = 150,  
    cv = 5,
    scoring = 'roc_auc',
    random_state = 42,
    n_jobs = -1
)

random_search.fit(X_res, y_res)

best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(filtered_test)[:, 1]

In [ ]:
test['TARGET'] = y_pred_proba
result = test[['SK_ID_CURR', 'TARGET']]

In [ ]:
result = result.sort_values(['SK_ID_CURR'])
result['SK_ID_CURR'] = result['SK_ID_CURR'].astype(int)
result

In [ ]:
result.to_csv('submit_7_nonmerge_lr_woe.csv', index = False)